In [2]:
cd "/home/sage/Documents/works/12. Developments/07. sage/02. IdIso/"

/home/sage/Documents/works/12. Developments/07. sage/02. IdIso


#  <font color="green">Implementation of IdealToIsogeny algorithm</font>

## <font color="green">0. Prime Search</font>

In [17]:
load('primegen.sage')
find_prime(20, 5)

5 1
8 2
17 3
32 4
65 5
128 6
257 7
512 8
1025 9
2048 10
4097 11
8192 12
16385 13
32768 14
65537 15
131072 16
262145 17
524288 18
1048577 19


SystemError: Real Interval Field with 32 bits of precision returned a result with an error set

## <font color="green">1. Prerequisites</font>

### Load required libraries

In [3]:
load('quaternion_tools.sage')
load('isogeny.sage')
load('richelot.sage')
load('const.sage')

### Cornaccia algorithm

In [8]:
def cornacchia(d, m):
    try: 
        (x,y) = DiagonalQuadraticForm(QQ, [1,d]).solve(m)
        if not x.is_integer() or not y.is_integer() : raise Exception("No solution")
        return x, y
    except: raise Exception("No solution")

## <font color="green">2. Setting the lab environment</font>

### 1. Get a random ideal $I$ of norm $\ell^e$

In [9]:
I = GetRandomIdealElle()
assert(I.norm() == ell^6)

### 2. Extract the first ideal factor $I_1$
+ $I = I_1 \cdot I_2 \cdot ... \cdot I_k$ and $n(I_i) = \ell$

In [12]:
ellO0 = QA.ideal(O0.basis()).scale(ell)
I1 = AddIdeal(I, ellO0)

### 3. Compute corresponding isogeny $\phi_1$ to $I_1$
+ $ker(\phi_1) = ker(I_1)$

In [13]:
phi1_kernel_generator = left_O0_ideal_to_isogeny_kernel(I1)
phi1 = Isogeny(Fp(0), phi1_kernel_generator)
assert(I1.norm() == ell)

### 4. Get the codomain curve $E_1$ of $\phi_1$ and the dual isogeny $\hat{\phi}_1$

In [14]:
E1 = phi1.codomain_curve
phi1_dual = phi1.dual_isogeny()

### 5. Compute the right order $\mathcal{O}_1$ of $I_1$
+ $\mathcal{O}_1 \cong End(E_1)$

In [15]:
O1 = I1.right_order()

### 6. Extract the second ideal factor $I_2$
+ $\bar{I_1} I \cap \ell \mathcal{O}_1 = I_2$

In [16]:
I2 = I1.conjugate() * I
I2 = I2.scale(1/ell)
ellO1 = QA.ideal(O1.basis()).scale(ell)
I2 = AddIdeal(I2, ellO1)

assert(I2.norm() == ell and I2.left_order() == O1)

## <font color="green">3. Experiments</font>

+ Compute the generator $\alpha_2$ of $I_2$ such that $I_2 = \alpha_2 \mathcal{O_1} + \ell \mathcal{O_1}$
+ Hopefully $n(\alpha_2)$ is as small as possible
+ Note that $\alpha_2 \in I_2 \subset \mathcal{O}_1$

In [18]:
I2_generator = ideal_generator(I2)

+ Compute the reduced basis $\{o_{11}, o_{12}, o_{13}, o_{14}\}$ of $\mathcal{O_1}$

In [20]:
O1_basis = GetReducedBasis(O1.basis())

+ Try to find the $\beta \in \mathcal{O}_1$ such that $n(\alpha_2) + n(\beta) = 2^e | \sqrt{\#E_1(\mathbb{F}_{p^k})}$
+ $\mathrm{ord}_{2}(\#E_1(\mathbb{F}_{p^k})) \approx \mathrm{ord}_2(\#E_1(\mathbb{F}_{p^2}))=2 \times \mathrm{ord}_2(p+1)$ regardless of the extension size $k$
    + So it must be satisfied that $2^e < (p+1)$
+ Find the coefficients $\{c_1, c_2, c_3, c_4\}$ such that $n(\beta) = n(\sum_{i}{c_i o_{1i}}) = 2^e - n(\alpha_2)$

In [35]:
testFp = GF(prime^16)
testE0 = EllipticCurve(testFp, [0, 0, 0, 1, 0])
print(testE0.cardinality().factor())
print(((prime + 1)^2).factor())

2^42 * 3^2 * 5^2 * 17^2 * 37^2 * 313^2 * 9769^2 * 218453^2 * 23216004013^2 * 28389975360007889^2
2^36 * 5^2
